In [14]:
from fastatomography.util import *
from fastatomography.tomo import ray_transforms
import numpy as np
from fastatomography.fasta_tomography import fasta_tomography_nonnegative_shrink
import os
from scipy.io import loadmat
import h5py
from pathlib import Path

base = "global/homes/l/ljoyce/"   # forward slashes always safe on Windows
# fn_path = base / "images.mat"
# angles_fn_path = base / "tiltangles.mat"

fn_path =Path("images.mat")
angles_fn_path = Path("tiltangles.mat")

# test if files exist
# print(fn_path)
# print(fn_path.exists())
# print(angles_fn_path)
# print(angles_fn_path.exists())

proj = loadmat('images')["stackNorm"]
angles_in = np.deg2rad(loadmat("tiltangles.mat")['tiltdegrees'])

#%%
print(proj.shape)
y = th.as_tensor(np.transpose(proj, (1,2,0)), dtype=th.float32).contiguous().cuda()
print(y.shape)
angles_in = th.as_tensor(angles_in.T).squeeze().contiguous().cuda()
print("angles_in shape:", angles_in.shape)


ps = proj.shape
projection_shape = [450, 450]
real_space_extent = np.array([450, 450, 450])
n_angles = angles_in.shape[0]
angles = th.zeros((3, n_angles))
angles[0, :] = 0
angles[1, :] = angles_in
angles[2, :] = 0

A, At = ray_transforms(real_space_extent, projection_shape, n_angles)

print("A geometry domain:", A.domain.shape)
print("A geometry range:", A.range.shape) #currently outputing A geometry range: (5, 5, 5, 450, 450)

x0 = th.zeros(projection_shape[0], projection_shape[1], projection_shape[1], dtype=th.float32).cuda()


#%%
mu = 1e-2
opts = Param()
opts.record_objective = True
opts.verbose = True
opts.string_header = '     '
opts.max_iters = 100
opts.tol = 1e-6
opts.accelerate = True
opts.adaptive = True

sol, out, opts = fasta_tomography_nonnegative_shrink(A, At, x0, y, mu, angles, opts)
np.save( base +'fasta.npy', sol.cpu().numpy())
#%%
# plot(sol[160,:,:].cpu())
for i in range(160,170):
    plot(sol[:,i,:].cpu())
# plot(sol[:,:,160].cpu())
#%%


(450, 450, 70)
torch.Size([450, 70, 450])
angles_in shape: torch.Size([70])
angle_partition uniform_partition([-0.1963, -0.0011, -0.0011], [ 1.7671,  0.0098,  0.0098], (5, 5, 5))
detector_partition uniform_partition([-225., -225.], [ 225.,  225.], (450, 450))
geometry Parallel3dEulerGeometry(
    uniform_partition([-0.1963, -0.0011, -0.0011], [ 1.7671,  0.0098,  0.0098], (5, 5, 5)),
    uniform_partition([-225., -225.], [ 225.,  225.], (450, 450))
)
angle_partition_dummy uniform_partition([-225., -225., -225.], [ 225.,  225.,  225.], (70, 450, 450))
RayBackProjection init
self.domain.real_space uniform_discr([-225., -225., -225.], [ 225.,  225.,  225.], (450, 450, 450), dtype='float32')
self.range.real_space  uniform_discr([-225., -225., -225.], [ 225.,  225.,  225.], (70, 450, 450), dtype='float32')
A geometry domain: (450, 450, 450)
A geometry range: (5, 5, 5, 450, 450)


ValueError: The dimensions of the data (450, 450, 450) do not match those specified in the geometry (70, 450, 450)